In [16]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_batch


# Loading processed CSV to notebook

In [8]:
review_processed = pd.read_csv("../data/processed/reviews_processed.csv")

review_processed.head()


,review_id,review_text,rating,review_date,review_year,review_month,bank_code,bank_name,user_name,thumbs_up,text_length,source
0,809c46d2-730e-446a-9061-2a45e978ad9d,good jop,5,2025-11-27,2025,11,BOA,Bank of Abyssinia,Yohanis Fikadu,0,8,Google Play
1,63f63b8b-016a-4262-a27e-f0de9ebf985a,ok,5,2025-11-27,2025,11,BOA,Bank of Abyssinia,Butu Mare,0,2,Google Play
2,f28a3a3c-eb94-4aab-88d2-89bcecebcc7b,bad exprience...it is so crushed,1,2025-11-27,2025,11,BOA,Bank of Abyssinia,ሻንበል ኪዳነ,0,32,Google Play
3,4ed89e8c-16dc-4763-94ca-04d05cf799a5,not user friendly at all it requires a huge co...,1,2025-11-26,2025,11,BOA,Bank of Abyssinia,Tsegaw Fikru,0,85,Google Play
4,f5bfd792-bc81-4d06-99c9-06c7743d3e09,ምንም የማይ ሰራ,1,2025-11-26,2025,11,BOA,Bank of Abyssinia,Elsa Tesfay,0,10,Google Play


In [9]:
review_processed.shape

(1800, 12)

# Connecting to PostgreSQL from notebook

In [10]:
# 1. Open a connection to your PostgreSQL DB
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="bank_user",          # the new user you created
    password="bank_user_123"  # <-- replace this
)

# 2. Create a cursor to run SQL
cur = conn.cursor()

# 3. Quick sanity check: count rows in banks table
cur.execute("SELECT bank_id, bank_code, bank_name, app_name FROM banks;")
rows = cur.fetchall()

for r in rows:
    print(r)

cur.close()
conn.close()
print("Connection OK.")


(1, 'CBE', 'Commercial Bank of Ethiopia', 'CBE App')
(2, 'BOA', 'Bank of Abyssinia', 'BOA App')
(3, 'DASHEN', 'Dashen Bank', 'Dashen App')
Connection OK.


# Mapping bank_code and bank_id

In [13]:
bank_map = {
    "CBE": 1,
    "BOA": 2,
    "DASHEN": 3
}

# Create a new column bank_id
review_processed["bank_id"] = review_processed["bank_code"].map(bank_map)

review_processed[["bank_code", "bank_id"]].head()


,bank_code,bank_id
0,BOA,2
1,BOA,2
2,BOA,2
3,BOA,2
4,BOA,2


# Clean dataframe for DB insert

In [15]:

# 1) Keeping only the columns needed
reviews_for_db = review_processed[[
    "review_id",
    "bank_id",
    "review_text",
    "rating",
    "review_date",
    "source"
]].copy()

# 2) Drop rows where bank_id is missing
reviews_for_db = reviews_for_db.dropna(subset=["bank_id", "review_text", "rating", "review_date", "source"])

# 3) Make sure the dtypes are correct
reviews_for_db["bank_id"] = reviews_for_db["bank_id"].astype(int)
reviews_for_db["rating"] = reviews_for_db["rating"].astype(int)

# 4) Check
reviews_for_db.head(), reviews_for_db.shape


(                              review_id  bank_id  \
 0  809c46d2-730e-446a-9061-2a45e978ad9d        2   
 1  63f63b8b-016a-4262-a27e-f0de9ebf985a        2   
 2  f28a3a3c-eb94-4aab-88d2-89bcecebcc7b        2   
 3  4ed89e8c-16dc-4763-94ca-04d05cf799a5        2   
 4  f5bfd792-bc81-4d06-99c9-06c7743d3e09        2   
 
                                          review_text  rating review_date  \
 0                                           good jop       5  2025-11-27   
 1                                                 ok       5  2025-11-27   
 2                   bad exprience...it is so crushed       1  2025-11-27   
 3  not user friendly at all it requires a huge co...       1  2025-11-26   
 4                                         ምንም የማይ ሰራ       1  2025-11-26   
 
         source  
 0  Google Play  
 1  Google Play  
 2  Google Play  
 3  Google Play  
 4  Google Play  ,
 (1800, 6))

# Insert reviews into reviews table

In [18]:
# 1) Open connection with the safe user
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="bank_user",
    password="bank_user_123",
)

cur = conn.cursor()

insert_sql = """
INSERT INTO reviews (
    review_id,
    bank_id,
    review_text,
    rating,
    review_date,
    source
)
VALUES (%s, %s, %s, %s, %s, %s)
ON CONFLICT (review_id) DO NOTHING;
"""

# 2) Convert DataFrame rows into list of tuples
records = [
    (
        row["review_id"],          
        int(row["bank_id"]),
        row["review_text"],
        int(row["rating"]),
        row["review_date"],        
        row["source"],
    )
    for _, row in reviews_for_db.iterrows()
]

# 3) Bulk insert in batches
execute_batch(cur, insert_sql, records, page_size=500)

conn.commit()
cur.close()
conn.close()

print(f"Inserted {len(records)} reviews into PostgreSQL.")


Inserted 1800 reviews into PostgreSQL.


# Run SQL checks

In [19]:
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="bank_user",
    password="bank_user_123",
)

def run_query(sql):
    return pd.read_sql(sql, conn)

# 1) Check total reviews inserted
sql = "SELECT COUNT(*) AS total_reviews FROM reviews;"
print("TOTAL REVIEWS:")
display(run_query(sql))

# 2) Check count per bank
sql = """
SELECT b.bank_name, COUNT(*) AS review_count
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name
ORDER BY review_count DESC;
"""
print("REVIEWS PER BANK:")
display(run_query(sql))

# 3) Average rating per bank
sql = """
SELECT b.bank_name, ROUND(AVG(r.rating), 2) AS avg_rating
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name
ORDER BY avg_rating DESC;
"""
print("AVERAGE RATING PER BANK:")
display(run_query(sql))

conn.close()


TOTAL REVIEWS:


C:\Users\yamis\AppData\Local\Temp\ipykernel_13384\2892611751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,total_reviews
0,1800


REVIEWS PER BANK:


C:\Users\yamis\AppData\Local\Temp\ipykernel_13384\2892611751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,bank_name,review_count
0,Commercial Bank of Ethiopia,600
1,Dashen Bank,600
2,Bank of Abyssinia,600


AVERAGE RATING PER BANK:


C:\Users\yamis\AppData\Local\Temp\ipykernel_13384\2892611751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


,bank_name,avg_rating
0,Dashen Bank,4.17
1,Commercial Bank of Ethiopia,4.13
2,Bank of Abyssinia,3.15
